In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

In [3]:
df_imdb = pd.read_csv('2016_2019_USmovies.csv')
df_imdb.isna().sum()

tconst                         0
titleType                      0
primaryTitle                   0
originalTitle                  0
isAdult                        0
startYear                      0
endYear                        0
runtimeMinutes                 0
genres                         0
title                          0
check                          0
averageRating                  0
numVotes                       0
budget_LCY                  1011
gross_USA_USD                930
cumm_worldwide_gross_USD     629
dtype: int64

In [90]:
# http://www.omdbapi.com/?i=tt3896198&apikey=81f8cc38

base_url = "http://www.omdbapi.com/?i="
key1 = "&apikey=81f8cc38"
key2 = "&apikey=a7bcd2ed"
key3 = "&apikey=7af14aee"

In [72]:
url = base_url+df_imdb.tconst.iloc[0]+end_url
r = get(url)
print(r.status_code)
response = r.json()
type(r.status_code)

200


int

In [11]:
response

{'Title': 'The Other Side of the Wind',
 'Year': '2018',
 'Rated': 'R',
 'Released': '02 Nov 2018',
 'Runtime': '122 min',
 'Genre': 'Drama',
 'Director': 'Orson Welles',
 'Writer': 'Orson Welles, Oja Kodar',
 'Actors': 'John Huston, Oja Kodar, Peter Bogdanovich, Susan Strasberg',
 'Plot': 'A Hollywood director emerges from semi-exile with plans to complete work on an innovative motion picture.',
 'Language': 'English, German',
 'Country': 'France, Iran, USA',
 'Awards': 'N/A',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.9/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '83%'},
  {'Source': 'Metacritic', 'Value': '78/100'}],
 'Metascore': '78',
 'imdbRating': '6.9',
 'imdbVotes': '4,936',
 'imdbID': 'tt0069049',
 'Type': 'movie',
 'DVD': '02 Nov 2018',
 'BoxOffice': 'N/A',
 'Production': 'Netflix',
 'Website': 'N/A',
 'Response': 'True'}

In [51]:
import datetime

date_string = response["Released"]
datetime.datetime.strptime(date_string, '%d %b %Y').month
str(datetime.datetime.strptime(date_string, '%d %b %Y').date())

'2018-11-02'

In [27]:
for i in response["Ratings"]:
    if list(i.values())[0] == "Rotten Tomatoes":
        print(str(list(i.values())[1]).replace("%", ""))
    else:
        continue

83


In [28]:
response["Metascore"]

'78'

In [29]:
response["Actors"]

'John Huston, Oja Kodar, Peter Bogdanovich, Susan Strasberg'

In [30]:
response["Production"]

'Netflix'

In [31]:
import pymongo
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
db = myclient["Project3"]
movie = db["Movies"]

In [79]:
#movie.insert(response)
movie.delete_many({})

In [65]:
query = movie.find({})

In [66]:
for line in query:
    print(line)

In [53]:
df_imdb["Released_Date"] = pd.Series([])
df_imdb["Released_Month"] = pd.Series([])
df_imdb["Rotten_Tomatoes"] = pd.Series([])
df_imdb["Metascore"] = pd.Series([])
df_imdb["Actors"] = pd.Series([])
df_imdb["Production"] = pd.Series([])

In [91]:
for i in range(200, 2015):
    url = base_url+df_imdb.tconst.iloc[i]+key2
    r = get(url)
    if r.status_code == 200:
        response = r.json()
        movie.insert(response)
        df_imdb["Released_Date"].iloc[i] = response["Released"]
        df_imdb["Metascore"].iloc[i] = response["Metascore"]
        df_imdb["Actors"].iloc[i] = response["Actors"]
        df_imdb["Production"].iloc[i] = response["Production"]
        for j in response["Ratings"]:
            if list(j.values())[0] == "Rotten Tomatoes":
                df_imdb["Rotten_Tomatoes"].iloc[i] = list(j.values())[1]
            else:
                continue
    else:
        continue

/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


In [ ]:
# for i in range(1000):
#     url = base_url+df_imdb.tconst.iloc[i]+key1
#     r = get(url)
#     if r.status_code == 200:
#         response = r.json()
#         movie.insert(response)
#     else:
#         continue

In [ ]:
# df_imdb["Released_Date"].iloc[i] = str(datetime.datetime.strptime(response["Released"], '%d %b %Y').date()) if response["Released"] != "N/A" else "N/A"
# df_imdb["Released_Month"].iloc[i] = datetime.datetime.strptime(response["Released"], '%d %b %Y').month if response["Released"] != "N/A" else "N/A"
# df_imdb["Metascore"].iloc[i] = int(response["Metascore"]) if response["Metascore"] != "N/A" else "N/A"
# df_imdb["Actors"].iloc[i] = response["Actors"]
# df_imdb["Production"].iloc[i] = response["Production"]
# for j in response["Ratings"]:
#     if list(j.values())[0] == "Rotten Tomatoes":
#         df_imdb["Rotten_Tomatoes"].iloc[i] = int((list(j.values())[1]).replace("%", ""))
#     else:
#         continue

In [81]:
df_imdb["Released_Date"].iloc[i] = str(datetime.datetime.strptime(response["Released"], '%d %b %Y').date()) if response["Released"] != "N/A" else "N/A"
df_imdb["Released_Month"].iloc[i] = datetime.datetime.strptime(response["Released"], '%d %b %Y').month if response["Released"] != "N/A" else "N/A"
df_imdb["Metascore"].iloc[i] = int(response["Metascore"]) if response["Metascore"] != "N/A" else "N/A"
df_imdb["Actors"].iloc[i] = response["Actors"]
df_imdb["Production"].iloc[i] = response["Production"]
for j in response["Ratings"]:
    if list(j.values())[0] == "Rotten Tomatoes":
        df_imdb["Rotten_Tomatoes"].iloc[i] = int((list(j.values())[1]).replace("%", ""))
    else:
        continue

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2015 entries, 0 to 2014
Data columns (total 22 columns):
tconst                      2015 non-null object
titleType                   2015 non-null object
primaryTitle                2015 non-null object
originalTitle               2015 non-null object
isAdult                     2015 non-null int64
startYear                   2015 non-null int64
endYear                     2015 non-null object
runtimeMinutes              2015 non-null int64
genres                      2015 non-null object
title                       2015 non-null object
check                       2015 non-null int64
averageRating               2015 non-null float64
numVotes                    2015 non-null int64
budget_LCY                  1004 non-null object
gross_USA_USD               1085 non-null float64
cumm_worldwide_gross_USD    1386 non-null float64
Released_Date               1000 non-null object
Released_Month              1000 non-null object
Rotten_Tomato

In [110]:
last_batch = list(df_imdb[df_imdb.Production.isna()].index)
last_batch

[1289, 1444, 1655, 1852, 2009, 2010, 2011, 2012, 2013, 2014]

In [111]:
for i in last_batch:
    url = base_url+df_imdb.tconst.iloc[i]+key2
    r = get(url)
    if r.status_code == 200:
        response = r.json()
        movie.insert(response)
        df_imdb["Released_Date"].iloc[i] = response["Released"]
        df_imdb["Metascore"].iloc[i] = response["Metascore"]
        df_imdb["Actors"].iloc[i] = response["Actors"]
        df_imdb["Production"].iloc[i] = response["Production"]
        for j in response["Ratings"]:
            if list(j.values())[0] == "Rotten Tomatoes":
                df_imdb["Rotten_Tomatoes"].iloc[i] = list(j.values())[1]
            else:
                continue
    else:
        continue

/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


In [115]:
df_imdb["Released_Month"] = df_imdb["Released_Month"].map(lambda x: datetime.datetime.strptime(x, '%d %b %Y').month if x != "N/A" else "N/A")
df_imdb

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title,...,numVotes,budget_LCY,gross_USA_USD,cumm_worldwide_gross_USD,Released_Date,Released_Month,Rotten_Tomatoes,Metascore,Actors,Production
0,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,\N,122,Drama,The Other Side of the Wind,...,4989,NaN,NaN,NaN,02 Nov 2018,11,83%,78,"John Huston, Oja Kodar, Peter Bogdanovich, Sus...",Netflix
1,tt0339736,movie,The Evil Within,The Evil Within,0,2017,\N,98,Horror,The Evil Within,...,2572,$6000000,NaN,NaN,30 Aug 2017,8,100%,N/A,"Sean Patrick Flanery, Dina Meyer, Frederick Ko...",N/A
2,tt0360556,movie,Fahrenheit 451,Fahrenheit 451,0,2018,\N,100,"Drama,Sci-Fi,Thriller",Fahrenheit 451,...,15295,NaN,NaN,NaN,20 May 2018,5,32%,N/A,"Michael B. Jordan, Aaron Davis, Cindy Katz, Mi...",HBO Films
3,tt0365545,movie,Nappily Ever After,Nappily Ever After,0,2018,\N,98,"Comedy,Drama,Romance",Nappily Ever After,...,6718,NaN,NaN,NaN,21 Sep 2018,9,71%,63,"Sanaa Lathan, Ricky Whittle, Lyriq Bent, Lynn ...",Netflix
4,tt0369610,movie,Jurassic World,Jurassic World,0,2015,\N,124,"Action,Adventure,Sci-Fi",Jurassic World,...,549806,$150000000,652.27,1670.4,12 Jun 2015,6,72%,59,"Chris Pratt, Bryce Dallas Howard, Irrfan Khan,...",Universal Pictures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,tt9562694,movie,Alien Warfare,Alien Warfare,0,2019,\N,88,"Action,Sci-Fi",Alien Warfare,...,2003,NaN,NaN,NaN,05 Apr 2019,4,NaN,N/A,"Clayton Snyder, David B. Meadows, Daniel Washi...",N/A
2011,tt9664078,movie,American Son,American Son,0,2019,\N,90,Drama,American Son,...,2233,NaN,NaN,NaN,01 Nov 2019,11,47%,N/A,"Kerry Washington, Jeremy Jordan, Steven Pasqua...",Netflix
2012,tt9815714,movie,The Hard Way,The Hard Way,0,2019,\N,92,Action,The Hard Way,...,1717,$3000000,NaN,NaN,05 Mar 2019,3,NaN,N/A,"Michael Jai White, Luke Goss, Randy Couture, M...",Netflix
2013,tt9831136,movie,The Banana Splits Movie,The Banana Splits Movie,0,2019,\N,89,"Comedy,Horror,Sci-Fi",The Banana Splits Movie,...,1668,NaN,NaN,NaN,13 Aug 2019,8,67%,N/A,"Dani Kind, Steve Lund, Celina Martin, Finlay W...",N/A


In [116]:
df_imdb["Metascore"] = df_imdb["Metascore"].map(lambda x: int(x) if x != "N/A" else "N/A")
df_imdb

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title,...,numVotes,budget_LCY,gross_USA_USD,cumm_worldwide_gross_USD,Released_Date,Released_Month,Rotten_Tomatoes,Metascore,Actors,Production
0,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,\N,122,Drama,The Other Side of the Wind,...,4989,NaN,NaN,NaN,02 Nov 2018,11,83%,78,"John Huston, Oja Kodar, Peter Bogdanovich, Sus...",Netflix
1,tt0339736,movie,The Evil Within,The Evil Within,0,2017,\N,98,Horror,The Evil Within,...,2572,$6000000,NaN,NaN,30 Aug 2017,8,100%,N/A,"Sean Patrick Flanery, Dina Meyer, Frederick Ko...",N/A
2,tt0360556,movie,Fahrenheit 451,Fahrenheit 451,0,2018,\N,100,"Drama,Sci-Fi,Thriller",Fahrenheit 451,...,15295,NaN,NaN,NaN,20 May 2018,5,32%,N/A,"Michael B. Jordan, Aaron Davis, Cindy Katz, Mi...",HBO Films
3,tt0365545,movie,Nappily Ever After,Nappily Ever After,0,2018,\N,98,"Comedy,Drama,Romance",Nappily Ever After,...,6718,NaN,NaN,NaN,21 Sep 2018,9,71%,63,"Sanaa Lathan, Ricky Whittle, Lyriq Bent, Lynn ...",Netflix
4,tt0369610,movie,Jurassic World,Jurassic World,0,2015,\N,124,"Action,Adventure,Sci-Fi",Jurassic World,...,549806,$150000000,652.27,1670.4,12 Jun 2015,6,72%,59,"Chris Pratt, Bryce Dallas Howard, Irrfan Khan,...",Universal Pictures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,tt9562694,movie,Alien Warfare,Alien Warfare,0,2019,\N,88,"Action,Sci-Fi",Alien Warfare,...,2003,NaN,NaN,NaN,05 Apr 2019,4,NaN,N/A,"Clayton Snyder, David B. Meadows, Daniel Washi...",N/A
2011,tt9664078,movie,American Son,American Son,0,2019,\N,90,Drama,American Son,...,2233,NaN,NaN,NaN,01 Nov 2019,11,47%,N/A,"Kerry Washington, Jeremy Jordan, Steven Pasqua...",Netflix
2012,tt9815714,movie,The Hard Way,The Hard Way,0,2019,\N,92,Action,The Hard Way,...,1717,$3000000,NaN,NaN,05 Mar 2019,3,NaN,N/A,"Michael Jai White, Luke Goss, Randy Couture, M...",Netflix
2013,tt9831136,movie,The Banana Splits Movie,The Banana Splits Movie,0,2019,\N,89,"Comedy,Horror,Sci-Fi",The Banana Splits Movie,...,1668,NaN,NaN,NaN,13 Aug 2019,8,67%,N/A,"Dani Kind, Steve Lund, Celina Martin, Finlay W...",N/A


In [125]:
df_imdb["Rotten_Tomatoes"] = df_imdb["Rotten_Tomatoes"].map(lambda x: str(x).replace("%", "") if ((x != "N/A") & (x != "NaN")) else "N/A")
df_imdb

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title,...,numVotes,budget_LCY,gross_USA_USD,cumm_worldwide_gross_USD,Released_Date,Released_Month,Rotten_Tomatoes,Metascore,Actors,Production
0,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,\N,122,Drama,The Other Side of the Wind,...,4989,NaN,NaN,NaN,02 Nov 2018,11,83,78,"John Huston, Oja Kodar, Peter Bogdanovich, Sus...",Netflix
1,tt0339736,movie,The Evil Within,The Evil Within,0,2017,\N,98,Horror,The Evil Within,...,2572,$6000000,NaN,NaN,30 Aug 2017,8,100,N/A,"Sean Patrick Flanery, Dina Meyer, Frederick Ko...",N/A
2,tt0360556,movie,Fahrenheit 451,Fahrenheit 451,0,2018,\N,100,"Drama,Sci-Fi,Thriller",Fahrenheit 451,...,15295,NaN,NaN,NaN,20 May 2018,5,32,N/A,"Michael B. Jordan, Aaron Davis, Cindy Katz, Mi...",HBO Films
3,tt0365545,movie,Nappily Ever After,Nappily Ever After,0,2018,\N,98,"Comedy,Drama,Romance",Nappily Ever After,...,6718,NaN,NaN,NaN,21 Sep 2018,9,71,63,"Sanaa Lathan, Ricky Whittle, Lyriq Bent, Lynn ...",Netflix
4,tt0369610,movie,Jurassic World,Jurassic World,0,2015,\N,124,"Action,Adventure,Sci-Fi",Jurassic World,...,549806,$150000000,652.27,1670.4,12 Jun 2015,6,72,59,"Chris Pratt, Bryce Dallas Howard, Irrfan Khan,...",Universal Pictures
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,tt9562694,movie,Alien Warfare,Alien Warfare,0,2019,\N,88,"Action,Sci-Fi",Alien Warfare,...,2003,NaN,NaN,NaN,05 Apr 2019,4,nan,N/A,"Clayton Snyder, David B. Meadows, Daniel Washi...",N/A
2011,tt9664078,movie,American Son,American Son,0,2019,\N,90,Drama,American Son,...,2233,NaN,NaN,NaN,01 Nov 2019,11,47,N/A,"Kerry Washington, Jeremy Jordan, Steven Pasqua...",Netflix
2012,tt9815714,movie,The Hard Way,The Hard Way,0,2019,\N,92,Action,The Hard Way,...,1717,$3000000,NaN,NaN,05 Mar 2019,3,nan,N/A,"Michael Jai White, Luke Goss, Randy Couture, M...",Netflix
2013,tt9831136,movie,The Banana Splits Movie,The Banana Splits Movie,0,2019,\N,89,"Comedy,Horror,Sci-Fi",The Banana Splits Movie,...,1668,NaN,NaN,NaN,13 Aug 2019,8,67,N/A,"Dani Kind, Steve Lund, Celina Martin, Finlay W...",N/A


In [126]:
df_imdb.to_csv('2016_2019_USmovies_API.csv', index = False)